In [1]:
import findspark
findspark.init("/home/ubuntu/bigdata/apps/spark")

from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [2]:
sc.stop()
sc = SparkContext(appName="NetWordCountStateWindow")
ssc = StreamingContext(sc, 100)

### Es necesario tener HDFS arrancado, guarda información (sólo si no ponemos ruta local)

In [7]:
    # To maintain status info we need to use checkpoints. This way, if our
    # application fails it will be able to resume operations correctly
    # later on, using the updated information
    checkpointDir = "file:///home/ubuntu/bigdata/apps/spark/checkpoint"
    ssc.checkpoint(checkpointDir)  # create dir for checkpoint archives
    #ssc.checkpoint("/tmp/checkpoint")  # Necesario HDFS

    lines = ssc.socketTextStream("localhost", 9997)
    counts = lines.flatMap(lambda line: line.split("b"))\
              .map(lambda word: (word, 1))\
              .reduceByKeyAndWindow(lambda x,y: x+y, lambda x, y: x-y, 604800, 600)              
    
    counts.pprint()

    #ssc.start()
    #ssc.awaitTermination()

In [6]:
ssc.start()


-------------------------------------------
Time: 2021-03-27 17:19:24
-------------------------------------------
('hola', 3)
('adios', 1)

-------------------------------------------
Time: 2021-03-27 17:19:36
-------------------------------------------
('hola', 4)
('adios', 1)

-------------------------------------------
Time: 2021-03-27 17:19:48
-------------------------------------------
('hola', 4)
('adios', 1)

-------------------------------------------
Time: 2021-03-27 17:20:00
-------------------------------------------
('hola', 1)

-------------------------------------------
Time: 2021-03-27 17:20:12
-------------------------------------------

-------------------------------------------
Time: 2021-03-27 17:20:24
-------------------------------------------

-------------------------------------------
Time: 2021-03-27 17:20:36
-------------------------------------------
('hola', 2)
('adios', 1)

-------------------------------------------
Time: 2021-03-27 17:20:48
-------------

In [7]:
ssc.stop()
sc.stop()
